**Note:** This notebook is written in the Julia language, so the cells can't be
  executed in Google Colab. If you want to verify that the notebook works, I
  recommend [JuliaBox](https://juliabox.com/) or testing locally. The syntax is
  very similar to Python and MATLAB. Note in particular the dot syntax used to
  perform elementwise operations (`f.(x)` applies `f` to all elements of `x`),
  that indices start at 1 and that the last statement of all functions is returned
  automatically.

# **Lab 3: Iterative methods**
**Anders Ågren Thuné**

# **Abstract**

In this report, various iterative methods for solving systems of equations, both
linear and nonlinear, were implemented. The algorithms largely worked as
expected.

# **About the code**

A short statement on who is the author of the file, and if the code is
distributed under a certain license.

In [1]:
"""
DD2363 Methods in Scientific Computing,
KTH Royal Institute of Technology, Stockholm, Sweden.
"""

# Copyright (C) 2019
# Anders Ågren Thuné (athune@kth.se)
# Johan Hoffman (jhoffman@kth.se)

# Code written by Anders Ågren Thuné based on the template by Johan Hoffman.

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

"DD2363 Methods in Scientific Computing,\nKTH Royal Institute of Technology, Stockholm, Sweden.\n"

# **Set up environment**

In [213]:
using LinearAlgebra

# **Introduction**

Direct methods for solving linear systems of equations are often efficient and
easy to implement. However, when the system dimensions get large or when dealing
with nonlinear systems, they fall short. Instead, iterative methods that neither
require the matrix inverse nor a linear system can be put to use.

In general, iterative methods are based on the concept of *fixed point
iteration*, which is where an equation of the form $x^{(k+1)} = f(x^{(k)})$
converges to some fixed value $x$. When dealing with linear equations, this
takes the form $x^{(k+1)}=Mx^{(k)}+c$, and it can be shown that the iteration
converges when $\|M\| < 1$. The situation is complicated somewhat when dealing
with nonlinear equations, but under the assumption that the function whose roots
are to be found is *Lipschitz continous* in the interval (that is, there is a
constant $L_f$ such that for any $x$ and $y$ in the interval, $\|f(x)-f(y)\| <
L_f\|x-y\|$), a similar assertion can be made. This is not uncommon, as all $f
\in C^'$ are Lipschitz continous, for instance. A suitable stopping criterion
for an iterative method can often be set as the following:
$\frac{\|r^{(k)}\|}{\|b\|} < TOL$, where $r^{(k)}$ is the residual at step $k$,
for some tolerance $TOL$.

This report presents how these concepts, described in chapters
7 and 8 of the lecture notes, were used to implement the following methods:
- Jacobi iteration for solving of linear systems
- Gauss-Seidel iteration for solving of linear systems
- Newton's method for solving of nonlinear scalar equations
- GMRES iteration for solving of linear systems
- Newton's method for solving of nonlinear systems

# **Methods**

### **Jacobi and Gauss-Seidel iteration**

The Jacobi and Gauss-Seidel methods are fixed-point iterations for solving
linear systems of equations $Ax=b$ based on *matrix splitting*. In these
methods, the matrix $A$ is split into a sum $A = A_1 + A_2$, where $A_1$ is easy
to invert, giving the equation $A_1x = -A_2x + b$. The iteration is then
formulated as $A_1x^{(k+1)} = -A_2x^{(k)} + b$, or $x^{(k+1)} = Mx^{(k)} + c$,
with $M = -A_1^{-1}A_2, \quad c=b$.

In the Jacobi method, the splitting $A = D + (A - D)$ is used, where $D$ is a
diagonal matrix with the same diagonal elements as $A$.

In the Gauss-Seidel method, the splitting $A = L + (A-L)$ is used, where $L$ is
a lower triangular matrix with the same lower triangular part as $A$.

[Chapter 7.2 of the notes]

In [214]:
function jacobi(A, b, x_zero = zeros(length(b)), TOL = 1e-6)
    x = x_zero
    n = length(b)
    x_prev = zeros(n)
    norm_b = norm(b)
    res = [Inf for _ = 1:n]

    k = 1
    while norm(res)/norm_b >= TOL
        x_prev .= x
        for i = 1:n
            x[i] = 1/A[i,i]*(b[i]-sum(j == i ? 0.0 : A[i,j]*x_prev[j] for j = 1:n))
        end
        mul!(res, A, x)
        res .-= b
        k+=1
    end
    x
end

jacobi (generic function with 3 methods)

In [215]:
function gauss_seidel(A, b, x_zero = zeros(length(b)), TOL = 1e-6)
    x = x_zero
    n = length(b)
    norm_b = norm(b)
    res = [Inf for _ = 1:n]

    while norm(res)/norm_b >= TOL
        for i = 1:n
            x[i] = 1/A[i,i]*(b[i]-sum(j == i ? 0.0 : A[i,j]*x[j] for j = 1:n))
        end
        mul!(res, A, x)
        res .-= b
    end
    x
end

gauss_seidel (generic function with 3 methods)

### **Newton's method for scalar equations**

Newton's method is an iterative method for solving nonlinear equations, with
$x^{(k+1)}= x^{(k)} - f'(x)^{-1}f(x)$. In order to compute the derivative of the
function $f$, I use a finite difference with an arbitrarily chosen small $h$. The
implementation is based on Algorithm 13 of chapter 8.3 in the notes.

In [216]:
findiff(f, x, h = 1e-6) = (f(x+h)-f(x))/h

findiff (generic function with 2 methods)

In [217]:
function newton(f, x_zero = 0, TOL = 1e-6)
    x = x_zero
    while abs(f(x)) >= TOL
        x -= f(x)/findiff(f, x)
    end
    x
end

newton (generic function with 3 methods)

### **GMRES**

The key idea of the GMRES method is to solve the least squares problem
$\displaystyle \min_{x^{k} \in \mathcal{K}_k}\|b-Ax^{(k)}\|$, where
$\mathcal{K}_k$ is the Krylov subspace $\langle b, Ab, \dots,
A^{(k-1)}b\rangle$. However, since using the Krylov vectors as they are results
in an unstable method, the GMRES method orthogonalizes these basis vectors
first. The implementation is based on algorithms 9 & 10 in chapter 7.3 of the
lecture notes.

In [218]:
function gmres(A, b, TOL = 1e-6, kmax=150)
    n = length(b)
    norm_b = norm(b)
    x = v = zeros(n)

    k_capac = 10 # initial capacity
    Q = zeros(n,k_capac)
    H = zeros(k_capac+1,k_capac)
    b_e1 = [norm_b; zeros(k_capac-1)]
    y = zeros(k_capac)

    Q[:,1] .= b./norm_b
    res = b - Q*(A*Q \ b) # \ solves least squares
    norm_res0 = norm(res)

    k = 1
    while norm(res)/norm_res0 >= TOL && k < kmax
        if k == k_capac # allocate more space
            Q, Q_prev = zeros(n,k_capac*2), Q
            H, H_prev = zeros(k_capac*2+1,k_capac*2), H
            Q[:,1:k_capac] .= Q_prev
            H[1:k_capac+1,1:k_capac] .= H_prev
            k_capac *= 2
            b_e1 = [norm_b; zeros(k_capac-1)]
            y = zeros(k_capac)
        end

        mul!(v, A, Q[:,k]) # arnoldi
        for j = 1:k
            H[j,k] = dot(Q[:,j],v)
            v .-= H[j,k].*Q[:,j]
        end
        H[k+1,k] = norm(v)
        Q[:,k+1] .= v./H[k+1,k]

        y[1:k] .= H[1:k+1,1:k] \ b_e1[1:k+1]
        mul!(x, Q[:,1:k], y[1:k])
        mul!(res,A,x)
        res.-=b
        k += 1
    end
    x
end

gmres (generic function with 3 methods)

### **Newton's method for nonlinear systems**

This is practically the same as the scalar case; the only difference is that
the Jacobian is somewhat harder to obtain than the scalar derivative. Due to lack
of time, I take the Jacobian as an argument to the function instead of
computing it myself.

In [219]:
function newtonvec(f, δf, x_zero::Vector, TOL = 1e-6)
    x = x_zero
    while norm(f(x)) >= TOL
        x.-=δf(x)\f(x)
    end
    x
end

newtonvec (generic function with 5 methods)

# **Results**

This section presents tests verifying that the implemented functions work as
expected. Again due to lack of time, the tests are more limited than previous
weeks.

In [220]:
TOL = 1e-3

A1 = [2. 3; 2 6]
A2 = [7. 4; -3 3]
A3 = [2. 2; 3 -5]

x1 = [3.0, 2.5]
x2 = [-2.5, 2]
x3 = [1.5, 1.5]

b1 = A1*x1
b2 = A2*x2
b3 = A3*x3

# Jacobi
@assert norm(b1-A1*jacobi(A1,b1)) < TOL
@assert norm(x1-jacobi(A1,b1)) < TOL
@assert norm(b2-A2*jacobi(A2,b2)) < TOL
@assert norm(x2-jacobi(A2,b2)) < TOL
@assert norm(b3-A3*jacobi(A3,b3)) < TOL
@assert norm(x3-jacobi(A3,b3)) < TOL

# Gauss-Seidel
@assert norm(b1-A1*gauss_seidel(A1,b1)) < TOL
@assert norm(x1-gauss_seidel(A1,b1)) < TOL
@assert norm(b2-A2*gauss_seidel(A2,b2)) < TOL
@assert norm(x2-gauss_seidel(A2,b2)) < TOL
@assert norm(b3-A3*gauss_seidel(A3,b3)) < TOL
@assert norm(x3-gauss_seidel(A3,b3)) < TOL

# GMRES
@assert norm(b1-A1*gmres(A1,b1)) < TOL
@assert norm(x1-gmres(A1,b1)) < TOL
@assert norm(b2-A2*gmres(A2,b2)) < TOL
@assert norm(x2-gmres(A2,b2)) < TOL
@assert norm(b3-A3*gmres(A3,b3)) < TOL
@assert norm(x3-gmres(A3,b3)) < TOL

# Newton
@assert abs(newton(x -> x^2+2x+1, -2) + 1) < TOL
@assert abs(newton(x -> sin(2x), -1) + pi/2) < TOL

println("Tests passed!")

Tests passed!


# **Discussion**

All implementations work as expected; however, the GMRES function doesn't seem to
approximate the solution well for $k < n$, so there might be some bug in the code.

# **Refences**

- Hoffman, J. 2019. *Introduction to Scientific Computing*